In [ ]:
from pathlib import Path as PythonPath

import numpy as np
import pyarrow as pa
import pyarrow.fs as fs
import pyarrow.dataset as ds
import pyarrow.parquet as pq

from object_store.arrow import ArrowFileSystemHandler

table = pa.table({"a": range(10), "b": np.random.randn(10), "c": [1, 2] * 5})

base = PythonPath.cwd()
store = fs.PyFileSystem(ArrowFileSystemHandler(str(base.absolute())))
arrow_fs = fs.SubTreeFileSystem(str(base.absolute()), fs.LocalFileSystem())

pq.write_table(table.slice(0, 5), "data/data1.parquet", filesystem=store)
pq.write_table(table.slice(5, 10), "data/data2.parquet", filesystem=store)

dataset = ds.dataset("data", format="parquet", filesystem=store)


In [ ]:
from pathlib import Path as PythonPath

import numpy as np
import pyarrow as pa
import pyarrow.fs as fs
import pyarrow.dataset as ds
import pyarrow.parquet as pq

from object_store.arrow import ArrowFileSystemHandler

table = pa.table({"a": range(10), "b": np.random.randn(10), "c": [1, 2] * 5})

base = PythonPath.cwd()
store = ArrowFileSystemHandler(str(base.absolute()))

import pickle

with PythonPath("asd.pkl").open("wb") as handle:
    pickle.dump(store, handle)

with PythonPath("asd.pkl").open("rb") as handle:
    store_pkl = pickle.load(handle)

store_pkl.get_file_info(["asd.pkl"])

In [ ]:
from object_store import ObjectStore, ObjectMeta

# we use an in-memory store for demonstration purposes.
# data will not be persisted and is not shared across store instances
store = ObjectStore("memory://")

store.put("data", b"some data")

data = store.get("data")
assert data == b"some data"

blobs = store.list()

meta: ObjectMeta = store.head("data")

range = store.get_range("data", start=0, length=4)
assert range == b"some"

store.copy("data", "copied")
copied = store.get("copied")
assert copied == data

In [ ]:
import duckdb

con = duckdb.connect()
results = con.execute("SELECT * FROM dataset WHERE c = 2").arrow()

results.shape

In [ ]:
visited_paths = []


def file_visitor(written_file):
    visited_paths.append(written_file)


partitioning = ds.partitioning(pa.schema([("c", pa.int64())]), flavor="hive")
ds.write_dataset(
    table,
    "partitioned",
    partitioning=partitioning,
    format="parquet",
    filesystem=store,
    file_visitor=file_visitor,
)

len(visited_paths)


In [ ]:
partitioning = ds.partitioning(pa.schema([("c", pa.int64())]), flavor="hive")
dataset_part = ds.dataset("/partitioned", format="parquet", filesystem=store, partitioning=partitioning)
dataset_part.schema
